# Portfolio Options Pricing System - Interactive Analysis

This notebook demonstrates the key features of the portfolio options pricing system.

In [ ]:
# Import required libraries
import sys
import os
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from src.models.black_scholes import BlackScholesModel
from src.portfolio.optimizer import PortfolioOptimizer
from src.data.market_data import MarketDataFetcher
from src.analysis.performance import PerformanceAnalyzer
from src.utils.helpers import format_percentage, format_currency

## 1. Black-Scholes Option Pricing

In [ ]:
# Initialize Black-Scholes model
bs_model = BlackScholesModel()

# Option parameters
S = 100    # Current stock price
K = 105    # Strike price
T = 0.25   # Time to expiration (3 months)
r = 0.05   # Risk-free rate
sigma = 0.2  # Volatility

# Calculate option prices
call_price = bs_model.calculate_price(S, K, T, r, sigma, 'call')
put_price = bs_model.calculate_price(S, K, T, r, sigma, 'put')

print(f"Call Option Price: {format_currency(call_price)}")
print(f"Put Option Price: {format_currency(put_price)}")

In [ ]:
# Calculate and display Greeks
greeks = bs_model.calculate_greeks(S, K, T, r, sigma, 'call')

print("Option Greeks:")
for greek, value in greeks.items():
    print(f"  {greek.capitalize()}: {value:.4f}")

## 2. Portfolio Optimization

In [ ]:
# Create synthetic return data for demonstration
symbols = ['AAPL', 'GOOGL', 'MSFT', 'TSLA']
dates = pd.date_range(start='2023-01-01', end='2023-12-31', freq='D')

np.random.seed(42)
synthetic_returns = {}
for symbol in symbols:
    returns = np.random.normal(0.001, 0.02, len(dates))
    synthetic_returns[symbol] = returns

returns_df = pd.DataFrame(synthetic_returns, index=dates)
print(f"Generated returns data shape: {returns_df.shape}")
returns_df.head()

In [ ]:
# Optimize portfolio
optimizer = PortfolioOptimizer(risk_free_rate=0.02)
optimal_portfolio = optimizer.optimize_portfolio(returns_df, optimization_target='sharpe')

print("Optimal Portfolio (Max Sharpe Ratio):")
print(f"  Expected Return: {format_percentage(optimal_portfolio.expected_return)}")
print(f"  Volatility: {format_percentage(optimal_portfolio.volatility)}")
print(f"  Sharpe Ratio: {optimal_portfolio.sharpe_ratio:.3f}")

print("\nOptimal Weights:")
weights_df = pd.DataFrame({
    'Symbol': optimal_portfolio.symbols,
    'Weight': optimal_portfolio.weights
})
weights_df['Weight (%)'] = weights_df['Weight'].apply(lambda x: f"{x*100:.2f}%")
weights_df

## 3. Performance Analysis

In [ ]:
# Analyze portfolio performance
analyzer = PerformanceAnalyzer()
portfolio_returns = (returns_df * optimal_portfolio.weights).sum(axis=1)

metrics = analyzer.calculate_performance_metrics(portfolio_returns)

print("Portfolio Performance Metrics:")
for metric, value in metrics.items():
    if 'ratio' in metric.lower():
        print(f"  {metric.replace('_', ' ').title()}: {value:.3f}")
    else:
        print(f"  {metric.replace('_', ' ').title()}: {format_percentage(value)}")

In [ ]:
# Plot cumulative returns
cumulative_returns = (1 + returns_df).cumprod()
portfolio_cumulative = (1 + portfolio_returns).cumprod()

plt.figure(figsize=(12, 6))
for symbol in symbols:
    plt.plot(cumulative_returns.index, cumulative_returns[symbol], label=symbol, alpha=0.7)
plt.plot(portfolio_cumulative.index, portfolio_cumulative, label='Optimized Portfolio', linewidth=2, color='black')

plt.title('Cumulative Returns Comparison')
plt.xlabel('Date')
plt.ylabel('Cumulative Return')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 4. Monte Carlo Simulation

In [ ]:
# Run Monte Carlo simulation
simulation_results = optimizer.monte_carlo_simulation(
    returns_df, 
    optimal_portfolio.weights,
    initial_investment=10000,
    time_horizon=252,  # 1 year
    num_simulations=1000
)

print("Monte Carlo Simulation Results (1 Year, $10,000 initial):")
for percentile, value in simulation_results['percentiles'].items():
    print(f"  {percentile} Percentile: {format_currency(value)}")

In [ ]:
# Plot simulation results
plt.figure(figsize=(12, 8))

# Plot sample trajectories
plt.subplot(2, 1, 1)
for i in range(min(100, simulation_results['simulations'].shape[0])):
    plt.plot(simulation_results['simulations'][i], alpha=0.1, color='lightblue')

# Plot percentiles
percentiles = np.percentile(simulation_results['simulations'], [5, 50, 95], axis=0)
plt.plot(percentiles[1], color='red', linewidth=2, label='Median')
plt.plot(percentiles[0], color='orange', linewidth=2, label='5th Percentile')
plt.plot(percentiles[2], color='green', linewidth=2, label='95th Percentile')

plt.title('Portfolio Value Trajectories (Monte Carlo Simulation)')
plt.xlabel('Days')
plt.ylabel('Portfolio Value ($)')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot final value distribution
plt.subplot(2, 1, 2)
plt.hist(simulation_results['final_values'], bins=50, alpha=0.7, color='lightblue', edgecolor='black')
plt.axvline(simulation_results['percentiles']['50th'], color='red', linestyle='--', label='Median')
plt.axvline(simulation_results['percentiles']['5th'], color='orange', linestyle='--', label='5th Percentile')
plt.axvline(simulation_results['percentiles']['95th'], color='green', linestyle='--', label='95th Percentile')

plt.title('Distribution of Final Portfolio Values')
plt.xlabel('Final Portfolio Value ($)')
plt.ylabel('Frequency')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()